# 데이터 전처리(Literacy)

---

1. 개봉일 데이터와 merge
2. 흥행영화 추출
3. 배급사(국내, 상장, 주요) 더미화, 연령 더미화
4. 배우, 감독 가중치 파생변수 생성

## 라이브러리 및 데이터 로드

In [1]:
## 라이브러리 로드
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
import os

# 경고 메세지 삭제
import warnings
warnings.filterwarnings('ignore')

# 한글 깨짐 방지
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'

# pd.set_option('display.max_rows', None)

---
# 1. 개봉일 데이터와 merge

In [2]:
## 데이터 로드
df = pd.read_excel('./datasets/movie_1.xlsx')
df

,순번,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국,전국.1,전국.2,서울,서울.1,장르,등급,영화구분
0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,스크린수,매출액,관객수,매출액,관객수,NaN,NaN,NaN
1,1.0,명량,김한민,(주)빅스톤픽쳐스,NaN,(주)씨제이이엔엠,2014-07-30,개봉영화,장편,한국,1587,135748398910,17613682,33121225810,4163666,사극,15세이상관람가,일반영화
2,2.0,극한직업,이병헌,"(주)어바웃잇,영화사 해그림 주식회사,(주)씨제이이엔엠",NaN,(주)씨제이이엔엠,2019-01-23,개봉영화,장편,한국,1978,139647979516,16264944,31858660536,3638287,코미디,15세이상관람가,일반영화
3,3.0,신과함께-죄와 벌,김용화,"리얼라이즈픽쳐스(주),(주)덱스터스튜디오",NaN,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20,개봉영화,장편,한국,1912,115698654137,14410754,27530825087,3346172,판타지,12세이상관람가,일반영화
4,4.0,국제시장,윤제균,"(주)제이케이필름,(주)씨제이이엔엠",NaN,(주)씨제이이엔엠,2014-12-17,개봉영화,장편,한국,966,110828014630,14245998,25842519330,3233946,드라마,12세이상관람가,일반영화
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11834,11834.0,VJ의 은밀한 사생활,오카다 히로,NaN,(주)영진크리에이티브,(주)영진크리에이티브,2017-12-25,개봉영화,장편,일본,1,6000,1,0,0,멜로/로맨스,청소년관람불가,일반영화
11835,11835.0,av감독의 아내,쿠도 마사노리,NaN,(주)도키엔터테인먼트,(주)도키엔터테인먼트,2016-12-22,개봉영화,장편,일본,1,8000,1,0,0,성인물(에로),청소년관람불가,일반영화
11836,11836.0,만화방,허재형,(주)영진크리에이티브,NaN,(주)영진크리에이티브,2012-02-29,개봉영화,장편,한국,0,0,0,0,0,성인물(에로),청소년관람불가,일반영화
11837,11837.0,클래쉬,레 탄 선,NaN,(주)소나무픽쳐스,(주)소나무픽쳐스,2011-08-11,개봉영화,장편,베트남,0,0,0,0,0,액션,청소년관람불가,일반영화


In [3]:
df['영화명'] = df['영화명'].astype('object')

In [4]:
# index 0번 삭제 및 순번 컬럼 삭제
df.drop('순번', axis=1, inplace=True, errors='ignore')
df.drop(index=0, inplace=True)

In [5]:
# 컬럼명 변경
df.rename(columns={'전국':'전국 스크린수', '전국.1':'전국 매출액', '전국.2': '전국 관객수', '서울':'서울 매출액', '서울.1':'서울 관객수' },
           inplace=True, errors='ignore')

In [6]:
## 영화명이 동일한 데이터 구분

# 나를 찾아줘 영화 구분
df[(df['영화명'] == '나를 찾아줘') & (df['개봉일'] == '2014-10-23')]
df.loc[(df['영화명'] == '나를 찾아줘') & (df['개봉일'] == '2014-10-23'), '영화명'] = '나를 찾아줘(외)'

# 스파이 영화 구분
df[(df['영화명'] == '스파이') & (df['개봉일'] == '2015-05-21')] 
df.loc[(df['영화명'] == '스파이') & (df['개봉일'] == '2015-05-21'), '영화명'] = '스파이(외)'

# 미녀와 야수1 영화 구분
df[(df['영화명'] == '미녀와 야수') & (df['개봉일'] == '2014-06-18')] 
df.loc[(df['영화명'] == '미녀와 야수') & (df['개봉일'] == '2014-06-18'), '영화명'] = '미녀와 야수1'

# 미녀와 야수2 영화 구분
df[(df['영화명'] == '미녀와 야수') & (df['개봉일'] == '2017-03-16')]
df.loc[(df['영화명'] == '미녀와 야수') & (df['개봉일'] == '2017-03-16'), '영화명'] = '미녀와 야수2'

In [7]:
## 개봉일 데이터 로드
daily_df = pd.read_csv('./datasets/Movie_Daily_Data.csv',index_col=0)
daily_df

,해당일,영화명,대표코드,해당일의 관객수,전일 대비 관객수 증감분,전일 대비 관객수 증감 비율,누적관객수,개봉일,당일 매출액,해당일자 매출총액 대비 해당 영화의 매출비율,전일 대비 매출액 증감분,전일 대비 매출액 증감 비율,누적매출액,스크린수,상영된 횟수,순위,전일대비 순위의 증감,신규진입여부
0,2019-05-29,0.0MHz,20188144,31078,29321,1668.8,33454,2019-05-29,194232300,5.4,178979300,1173.4,214496300,446,1043,4,8,OLD
1,2019-05-30,0.0MHz,20188144,10969,-20109,-64.7,44423,2019-05-29,90710800,1.5,-103521500,-53.3,305207100,375,723,5,-1,OLD
2,2019-05-31,0.0MHz,20188144,14921,3952,36.0,59344,2019-05-29,132797440,1.6,42086640,46.4,438004540,397,735,5,0,OLD
3,2019-06-01,0.0MHz,20188144,25945,11024,73.9,85289,2019-05-29,220796140,1.5,87998700,66.3,658800680,409,673,5,0,OLD
4,2019-06-02,0.0MHz,20188144,21792,-4153,-16.0,107081,2019-05-29,187628590,1.5,-33167550,-15.0,846429270,401,634,5,0,OLD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38688,2019-09-23,"힘을 내요, 미스터 리",20187322,12712,-31146,-71.0,1132202,2019-09-11,95046390,6.5,-278935910,-74.6,9595283006,558,1668,5,0,OLD
38689,2019-09-24,"힘을 내요, 미스터 리",20187322,13612,900,7.1,1145814,2019-09-11,100037300,6.1,4990910,5.3,9695320306,584,1642,6,-1,OLD
38690,2019-09-25,"힘을 내요, 미스터 리",20187322,5965,-7647,-56.2,1151779,2019-09-11,34873770,1.4,-65163530,-65.1,9730194076,256,359,10,-4,OLD
38691,2019-09-26,"힘을 내요, 미스터 리",20187322,3710,-2255,-37.8,1155489,2019-09-11,24129260,1.3,-10744510,-30.8,9754323336,215,305,10,0,OLD


In [8]:
# 개봉일 2010년 이후 데이터만 추출
daily_df = daily_df[daily_df['개봉일'] > '2010-01-01']

In [9]:
daily_df['영화명'] = daily_df['영화명'].astype('object')

In [10]:
daily_df = daily_df[daily_df['해당일'] == daily_df['개봉일']]

In [11]:
daily_df['개봉일'] = pd.to_datetime(daily_df['개봉일'])

In [12]:
## 영화명이 동일한 데이터 구분

# 나를 찾아줘 영화 구분
daily_df[(daily_df['영화명'] == '나를 찾아줘') & (daily_df['개봉일'] == '2014-10-23')]
daily_df.loc[(daily_df['영화명'] == '나를 찾아줘') & (daily_df['개봉일'] == '2014-10-23'), '영화명'] = '나를 찾아줘(외)'

# 스파이 영화 구분
daily_df[(daily_df['영화명'] == '스파이') & (daily_df['개봉일'] == '2015-05-21')] 
daily_df.loc[(daily_df['영화명'] == '스파이') & (daily_df['개봉일'] == '2015-05-21'), '영화명'] = '스파이(외)'

# 미녀와 야수1 영화 구분
daily_df[(daily_df['영화명'] == '미녀와 야수') & (daily_df['개봉일'] == '2014-06-18')] 
daily_df.loc[(daily_df['영화명'] == '미녀와 야수') & (daily_df['개봉일'] == '2014-06-18'), '영화명'] = '미녀와 야수1'

# 미녀와 야수2 영화 구분
daily_df[(daily_df['영화명'] == '미녀와 야수') & (daily_df['개봉일'] == '2017-03-16')]
daily_df.loc[(daily_df['영화명'] == '미녀와 야수') & (daily_df['개봉일'] == '2017-03-16'), '영화명'] = '미녀와 야수2'

In [13]:
daily_df['영화명'] = daily_df['영화명'].str.strip()

In [14]:
# 데이터 결합
df = pd.merge(df, daily_df, how='left', on=['영화명', '개봉일'])

---
# 2. 흥행영화 추출

In [15]:
# 연도만 추출하여 새로운 파생변수 개봉연도 생성
df['개봉일'] = pd.to_datetime(df['개봉일'])
df['개봉연도'] = df['개봉일'].dt.year

In [16]:
# 새로운 파생변수 평균 제작비 컬럼 생성 ( 한국영화진흥원 보고서 참고 )
def change(x):
    if x == 2009:
        result = 23.1
    elif x == 2010:
        result = 21.6
    elif x == 2011:
        result = 22.7
    elif x == 2012:
        result = 20.3
    elif x == 2013:
        result = 21.4
    elif x == 2014:
        result = 20.1
    elif x == 2015:
        result = 19.9
    elif x == 2016:
        result = 24.0
    elif x == 2017:
        result = 26.3
    elif x == 2018:
        result = 26.8
    elif x == 2019:
        result = 29.0
    else:
        result = '!'
    return result

df['평균 제작비'] = df['개봉연도'].apply(change)

In [17]:
# 매출액 억단위로 변경
df['수정전국매출액']  =  df['전국 매출액'] / 100000000

In [18]:
# 평균 제작비보다 매출액이 더 높은 데이터만 추출
df = df[df['수정전국매출액'] >= df['평균 제작비']]

In [19]:
# 영화명이 2개인 데이터 확인 -> 데이터 merge과정에서 중복 결합 확인 -> 모두 제거
df['영화명'].value_counts().head(14)

아쿠아맨                     2
도어락                      2
스윙키즈                     2
점박이 한반도의 공룡2 : 새로운 낙원    2
스파이더맨: 뉴 유니버스            2
PMC: 더 벙커                2
그린치                      2
범블비                      2
호두까기 인형과 4개의 왕국          2
국가부도의 날                  2
마약왕                      2
하늘에서 음식이 내린다면            1
고양이: 죽음을 보는 두 개의 눈       1
퍼스트맨                     1
Name: 영화명, dtype: int64

In [20]:
# 중복 영화 데이터 제거
df = df.drop_duplicates('영화명')

In [21]:
# 인덱스 초기화
df.reset_index(drop=True, inplace=True)

In [22]:
df.columns

Index(['영화명', '감독', '제작사', '수입사', '배급사', '개봉일', '영화유형', '영화형태', '국적',
       '전국 스크린수', '전국 매출액', '전국 관객수', '서울 매출액', '서울 관객수', '장르', '등급', '영화구분',
       '해당일', '대표코드', '해당일의 관객수', '전일 대비 관객수 증감분', '전일 대비 관객수 증감 비율', '누적관객수',
       '당일 매출액', '해당일자 매출총액 대비 해당 영화의 매출비율', '전일 대비 매출액 증감분',
       '전일 대비 매출액 증감 비율', '누적매출액', '스크린수', '상영된 횟수', '순위', '전일대비 순위의 증감',
       '신규진입여부', '개봉연도', '평균 제작비', '수정전국매출액'],
      dtype='object')

In [23]:
df = df[['영화명', '배급사', '개봉일', '국적', '전국 스크린수', '전국 매출액', '전국 관객수', '서울 매출액', '서울 관객수', 
    '장르', '등급', '영화구분', '개봉연도', '해당일의 관객수', '당일 매출액', '해당일자 매출총액 대비 해당 영화의 매출비율', '대표코드']]

In [24]:
df[df['대표코드'].isna()]

,영화명,배급사,개봉일,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분,개봉연도,해당일의 관객수,당일 매출액,해당일자 매출총액 대비 해당 영화의 매출비율,대표코드
39,1987,(주)씨제이이엔엠,2017-12-27,한국,1299,58165581145,7231638,14638920495,1776877,드라마,15세이상관람가,일반영화,2017,NaN,NaN,NaN,NaN
481,슈퍼 배드,유니버설픽쳐스인터내셔널 코리아(유),2010-09-16,미국,321,9873975412,1043904,2753597000,272176,애니메이션,전체관람가,일반영화,2010,NaN,NaN,NaN,NaN
776,"울지마, 톤즈",주식회사 마운틴픽쳐스,2010-09-09,한국,22,2529614776,441707,1084744000,174478,다큐멘터리,전체관람가,독립/예술영화,2010,NaN,NaN,NaN,NaN
829,내 이름은 칸,필라멘트픽쳐스,2011-03-24,인도,84,2820922500,381243,1210673500,157628,드라마,12세이상관람가,독립/예술영화,2011,NaN,NaN,NaN,NaN
848,극장판 짱구는 못말려: 아뵤! 쿵후 보이즈 ~라면 대란~,(주)씨제이이엔엠,2018-12-19,일본,489,2735399650,353012,383636800,47840,애니메이션,전체관람가,일반영화,2018,NaN,NaN,NaN,NaN


In [25]:
# 결측치 채우기
df.loc[df['영화명'] == 1987, '해당일의 관객수'] = 331736
df.loc[df['영화명'] == 1987, '당일 매출액'] = 1973650400
df.loc[df['영화명'] == 1987, '해당일자 매출총액 대비 해당 영화의 매출비율'] = 32.3
df.loc[df['영화명'] == 1987, '대표코드'] = 20170590

df.loc[df['영화명'] == '울지마, 톤즈', '해당일의 관객수'] = 2666
df.loc[df['영화명'] == '울지마, 톤즈', '당일 매출액'] = 17689500
df.loc[df['영화명'] == '울지마, 톤즈', '해당일자 매출총액 대비 해당 영화의 매출비율'] = 0.9
df.loc[df['영화명'] == '울지마, 톤즈', '대표코드'] = 20100962

df.loc[df['영화명'] == '내 이름은 칸', '해당일의 관객수'] = 14975
df.loc[df['영화명'] == '내 이름은 칸', '당일 매출액'] = 119445500
df.loc[df['영화명'] == '내 이름은 칸', '해당일자 매출총액 대비 해당 영화의 매출비율'] = 3.2
df.loc[df['영화명'] == '내 이름은 칸', '대표코드'] = 20119771

df.loc[df['영화명'] == '극장판 짱구는 못말려: 아뵤! 쿵후 보이즈 ~라면 대란~', '해당일의 관객수'] = 44580
df.loc[df['영화명'] == '극장판 짱구는 못말려: 아뵤! 쿵후 보이즈 ~라면 대란~', '당일 매출액'] = 348391550
df.loc[df['영화명'] == '극장판 짱구는 못말려: 아뵤! 쿵후 보이즈 ~라면 대란~', '해당일자 매출총액 대비 해당 영화의 매출비율'] = 3.3
df.loc[df['영화명'] == '극장판 짱구는 못말려: 아뵤! 쿵후 보이즈 ~라면 대란~', '대표코드'] = 20183479

In [26]:
# 대표코드가 없는 데이터 제거
df = df[~df['대표코드'].isna()]

In [27]:
df.reset_index(drop=True, inplace=True)

In [28]:
df['대표코드'] = df['대표코드'].astype(int)

In [29]:
# df.to_csv('./datasets/배우감독파생변수용.csv')

---
# 3. 더미화

### 국적, 영화등급, 영화구분 더미화

In [30]:
## 국적 컬럼 변경 (국내 : 1, 해외 : 0)

nation = {
    "한국": 1,
}

df['국적'] = df['국적'].map(nation).fillna(0)

In [31]:
df['국적'] = df['국적'].astype(int)

In [32]:
## 영화등급 컬럼 변경

grade = {
    "청소년관람불가": '청불',
    "15세이상관람가":  '15',
    "12세이상관람가":  '12',
    "전체관람가": '전체',
    "청소년관람불가,15세이상관람가" : '청불'
}

df['등급'] = df['등급'].map(grade)

In [33]:
dummies_genre = pd.get_dummies(df['등급'], prefix='등급')
df = pd.concat([df,dummies_genre], axis=1)

In [34]:
## 영화구분 분류 (1 : 일반영화, 0 : 독립/예술영화)

m = {
    "일반영화": 1,
    "독립/예술영화": 0
}

df['영화구분'] = df['영화구분'].map(m)

### 배급사 더미화
* 상장배급사 (상장 : 1, 비상장 : 0)
* 국내배급사 (국내 : 1, 해외 : 0)
* 주요배급사 (메이저 : 1, 비메이저 : 0)

In [35]:
# 배급사 컬럼만 추출
gg = df['배급사']

# 배급사 컬럼에서 외국 영화사명 포함된 values 추출 및 병합
a1 = gg[gg.str.contains("디즈니")]
a2 = gg[gg.str.contains("워너브러더스")]
a3 = gg[gg.str.contains("폭스")]
a4 = gg[gg.str.contains("소니")]
a5 = gg[gg.str.contains("유니버설")]
kkk = pd.concat([a1, a2, a3, a4, a5], axis=0)

# 새로 생성된 컬럼 범주화(0,1)

for i in kkk:
    for j in range(len(df)):
        if i in df['배급사'][j]:
            df.loc[j, '국내배급사'] = 1

ttt = df['국내배급사']
ttt.fillna(0, inplace=True)

# 배급사 컬럼에서 상장된 회사 추출 및 병합
b1 = gg[gg.str.contains("씨제이")]
b2 = gg[gg.str.contains("CJ")]
b3 = gg[gg.str.contains("CGV")]
b4 = gg[gg.str.contains("롯데")]
b5 = gg[gg.str.contains("NEW")]
b6 = gg[gg.str.contains("쇼박스")]
b7 = gg[gg.str.contains("스튜디오미르")]
b8 = gg[gg.str.contains("위지윅스튜디오")]
b9 = gg[gg.str.contains("덱스터")]
b10 = gg[gg.str.contains("samg엔터")]
b11= gg[gg.str.contains("애니플러스")]
b12 = gg[gg.str.contains("바른손이앤에이")]
b13 = gg[gg.str.contains("판타지오")]
b14= gg[gg.str.contains("스튜디오산타클로스")]
b15= gg[gg.str.contains("아센디오")]
b16 = gg[gg.str.contains("IHQ")]
b17= gg[gg.str.contains("콘텐트리중앙")]

kkkk = pd.concat([b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11, b12, b13, b14, b15, b16, b17], axis=0)

# 새로 생성된 컬럼 범주화(0,1)

for i in kkkk:
    for j in range(len(df)):
        if i in df['배급사'][j]:
            df.loc[j, '상장배급사'] = 1

df['상장배급사'].fillna(0, inplace=True)
ttttt = df['상장배급사']

# 배급사(메이저 비메이저) 컬럼 0,1로 변환(바이너리)후 새 컬럼에 삽입
df['배급사'] = df['배급사'].fillna('aa')

ls = ['이엔엠', 'CJ', '롯데', '쇼박스', '판다', '메가박스', '워너', '유니버설', '소니픽', '디즈니', '판씨', '키다리', 'NEW', '엣나인', '이수', '리틀', '스마일이', '팝엔', '오퍼스', '코리아스', '이십세기', '올스타']

for i in ls:
    for j in range(len(df)):
        if i in df['배급사'][j]:
            df.loc[:, '배급사'][j] = 'O'
            
df['주요배급사'] = df['배급사'].map(lambda x: 1 if x == 'O' else 0)

df['주요배급사'].value_counts()

# 배급사 컬럼 삭제
df = df.drop('배급사', axis=1)

---
# 4. 파생변수생성

### 배우가중치 
* (배우출현횟수 / 관객수가중치)
### 감독가중치
* (감독출현횟수 / 관객수가중치)

In [36]:
actor = pd.read_csv('./datasets/배우가중치.csv', index_col=0)
directors = pd.read_csv('./datasets/감독가중치.csv', index_col=0)

In [37]:
df = pd.concat([df, actor, directors], axis=1)

In [38]:
df['국내배급사'] = df['국내배급사'].astype(int)
df['상장배급사'] = df['상장배급사'].astype(int)
df['주요배급사'] = df['주요배급사'].astype(int)

In [39]:
df['배우가중치'] = round(df['배우가중치'],2)
df['감독가중치'] = round(df['감독가중치'],2)

In [40]:
df.columns

Index(['영화명', '개봉일', '국적', '전국 스크린수', '전국 매출액', '전국 관객수', '서울 매출액', '서울 관객수',
       '장르', '등급', '영화구분', '개봉연도', '해당일의 관객수', '당일 매출액',
       '해당일자 매출총액 대비 해당 영화의 매출비율', '대표코드', '등급_12', '등급_15', '등급_전체', '등급_청불',
       '국내배급사', '상장배급사', '주요배급사', '배우가중치', '감독가중치'],
      dtype='object')

In [41]:
df.columns = ['영화명', '개봉일', '국적', '전국스크린수', '전국매출액', '전국관객수', '서울매출액', '서울관객수',
       '장르', '등급', '영화구분', '개봉연도', '개봉일관객수', '개봉일매출액',
       '개봉일매출비율', '대표코드', '등급_12', '등급_15', '등급_전체', '등급_청불',
       '국내배급사', '상장배급사', '주요배급사', '배우가중치', '감독가중치']

In [42]:
df.drop(columns=['장르', '등급'], axis=1, inplace=True)

In [43]:
df

,영화명,개봉일,국적,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,영화구분,개봉연도,...,대표코드,등급_12,등급_15,등급_전체,등급_청불,국내배급사,상장배급사,주요배급사,배우가중치,감독가중치
0,명량,2014-07-30,1,1587,135748398910,17613682,33121225810,4163666,1,2014,...,20129370,0,1,0,0,0,1,1,39.00,2.00
1,극한직업,2019-01-23,1,1978,139647979516,16264944,31858660536,3638287,1,2019,...,20182530,0,1,0,0,0,1,1,21.00,3.00
2,신과함께-죄와 벌,2017-12-20,1,1912,115698654137,14410754,27530825087,3346172,1,2017,...,20150976,1,0,0,0,0,1,1,27.00,3.00
3,국제시장,2014-12-17,1,966,110828014630,14245998,25842519330,3233946,1,2014,...,20137048,1,0,0,0,0,1,1,29.00,1.00
4,어벤져스: 엔드게임,2019-04-24,0,2835,122182694160,13934592,33577136860,3597963,1,2019,...,20184889,1,0,0,0,1,0,1,22.00,3.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,스파이 브릿지,2015-11-05,0,425,2015360478,260721,722246378,90018,1,2015,...,20152871,1,0,0,0,1,0,1,0.33,0.10
913,함정,2015-09-10,1,445,2088685200,257716,411417600,50477,1,2015,...,20143510,0,0,0,1,0,0,0,0.63,0.03
914,파이널 데스티네이션5,2011-09-08,0,168,2370348500,257193,880006000,87499,1,2011,...,20112050,0,0,0,1,1,0,1,0.03,0.07
915,언브로큰,2015-01-07,0,401,1997947292,256475,605776392,75489,1,2015,...,20149948,0,1,0,0,1,0,1,0.23,0.03


In [45]:
df.to_csv('./datasets/1. 전처리완료.csv')